# 🤖 AI YouTube Shorts — Google Colab Quickstart

**Fully automated pipeline**: Topic → Story → SEO → 5 Scene Images → TTS → SadTalker Avatar → Subtitles → Slideshow → YouTube Upload

📦 **Repository**: [github.com/tamilarasu18/ai-yt-automation](https://github.com/tamilarasu18/ai-yt-automation)

### ☀️ Daily Routine
```
1. Open this notebook    → bookmark it!
2. Runtime → Run all     → 1 click
3. Wait ~15 minutes      → go drink coffee ☕
4. Telegram notification → "Video uploaded! ✅"
```

### 📋 Prerequisites
1. **Runtime → Change runtime type → T4 GPU**
2. Upload `config.json`, `service_account.json`, `client_secret.json` to `Drive/ai-youtube-automation/`
3. Google Sheet with topics (Status = `Pending`)


---
## 🔧 Step 1 — Mount Google Drive

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 1: Mount Google Drive (credentials only, no model cache)
# ═══════════════════════════════════════════════════════════════

from google.colab import drive
drive.mount('/content/drive')

import os

DRIVE_BASE = '/content/drive/MyDrive/ai-youtube-automation'

# Verify credentials
print('📁 Checking Google Drive...\n')
all_ok = True
for name in ['config.json', 'service_account.json', 'client_secret.json']:
    path = f'{DRIVE_BASE}/{name}'
    exists = os.path.exists(path)
    print(f'  {"✅" if exists else "❌"} {name}')
    if not exists: all_ok = False

if all_ok:
    print('\n✅ Drive ready!')
else:
    print('\n⚠️  Missing files! Upload them to Drive/ai-youtube-automation/')

---
## 📦 Step 2 — Clone Repo & Install Packages

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 2: Clone repo + install system deps + pip packages
# ═══════════════════════════════════════════════════════════════

import os

if not os.path.exists('/content/ai-yt-automation'):
    !git clone https://github.com/tamilarasu18/ai-yt-automation.git /content/ai-yt-automation
else:
    !cd /content/ai-yt-automation && git pull

os.chdir('/content/ai-yt-automation')

# System deps — zstd is REQUIRED by Ollama installer
!apt-get update -qq && apt-get install -y -qq ffmpeg espeak-ng zstd > /dev/null 2>&1

# Python deps
!pip install -e ".[dev,api,kokoro]" -q 2>/dev/null
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q 2>/dev/null
!pip install diffusers transformers accelerate safetensors -q 2>/dev/null
!pip install moviepy==1.0.3 -q 2>/dev/null
!pip install gspread google-auth google-auth-oauthlib google-api-python-client -q 2>/dev/null
!pip install python-telegram-bot==20.7 openai-whisper -q 2>/dev/null

print('\n✅ Packages installed!')

---
## 🤖 Step 3 — Setup Ollama + SadTalker

Downloads models to Colab's **local disk** (~40 GB available).  
Drive stays free — only credentials are stored there.

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 3: Setup Ollama + SadTalker (local disk, no Drive cache)
# ═══════════════════════════════════════════════════════════════

import os, subprocess, time

# ─── OLLAMA ───
print('🤖 Setting up Ollama...')
!curl -fsSL https://ollama.com/install.sh | sh

# Start Ollama server
for path in ['ollama', '/usr/local/bin/ollama']:
    try:
        subprocess.Popen([path, 'serve'],
                         stdout=subprocess.DEVNULL,
                         stderr=subprocess.DEVNULL,
                         start_new_session=True)
        break
    except FileNotFoundError:
        continue
time.sleep(5)

# Wait for server
import urllib.request
server_ok = False
for _ in range(10):
    try:
        urllib.request.urlopen('http://localhost:11434/api/tags', timeout=3)
        print('✅ Ollama server running!')
        server_ok = True
        break
    except Exception:
        time.sleep(2)

if server_ok:
    !ollama pull gemma3:4b
    print('✅ Ollama model ready!')
else:
    print('❌ Ollama server failed to start')

# ─── SADTALKER ───
print('\n🗣️ Setting up SadTalker...')
sadtalker_local = '/content/SadTalker'

if not os.path.exists(sadtalker_local):
    !git clone -q https://github.com/OpenTalker/SadTalker.git {sadtalker_local}

!sed -i 's/np.VisibleDeprecationWarning/DeprecationWarning/g' {sadtalker_local}/src/face3d/util/preprocess.py 2>/dev/null; true
!pip install -q face-alignment==1.3.5 kornia==0.6.12 yacs facexlib gfpgan basicsr dlib 2>/dev/null
!pip install -q -r {sadtalker_local}/requirements.txt 2>/dev/null; true

checkpoints_dir = f'{sadtalker_local}/checkpoints'
if not os.path.exists(checkpoints_dir) or not os.listdir(checkpoints_dir):
    print('⏳ Downloading SadTalker models...')
    !cd {sadtalker_local} && bash scripts/download_models.sh 2>/dev/null; true
print('✅ SadTalker ready!')

# ─── INFO ───
print('\n🖼️  SD Turbo (~2GB) will download on first pipeline run')

os.chdir('/content/ai-yt-automation')
print('\n' + '='*50)
print('✅ All models ready! (local disk, Drive stays free)')
!echo '\n📊 Disk:' && df -h /content | tail -1 | awk '{print "  " $3 " used / " $2 " (" $5 " full)"}'

---
## ⚙️ Step 4 — Load Config & Set Environment

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 4: Load config from Drive + set environment variables
# ═══════════════════════════════════════════════════════════════

import json, os, shutil

DRIVE_BASE = '/content/drive/MyDrive/ai-youtube-automation'
WORK_DIR = '/content/ai-yt-automation'

for fname in ['service_account.json', 'client_secret.json']:
    src = f'{DRIVE_BASE}/{fname}'
    dst = f'{WORK_DIR}/{fname}'
    if os.path.exists(src):
        shutil.copy(src, dst)
        print(f'  ✅ {fname}')
    else:
        print(f'  ⚠️  {fname} not found')

config = {}
config_path = f'{DRIVE_BASE}/config.json'
if os.path.exists(config_path):
    with open(config_path) as f:
        config = json.load(f)
    print(f'  ✅ config.json loaded')

env_map = {
    'google_sheet_url': 'GOOGLE_SHEET_URL',
    'google_sheet_name': 'GOOGLE_SHEET_NAME',
    'youtube_client_id': 'YOUTUBE_CLIENT_ID',
    'youtube_client_secret': 'YOUTUBE_CLIENT_SECRET',
    'youtube_refresh_token': 'YOUTUBE_REFRESH_TOKEN',
    'telegram_bot_token': 'TELEGRAM_BOT_TOKEN',
    'telegram_chat_id': 'TELEGRAM_CHAT_ID',
    'video_privacy': 'VIDEO_PRIVACY',
    'auto_upload_youtube': 'AUTO_UPLOAD_YOUTUBE',
}
for k, v in env_map.items():
    val = config.get(k, '')
    if val: os.environ[v] = str(val)

# Fixed Colab paths
os.environ['GOOGLE_SERVICE_ACCOUNT_FILE'] = f'{WORK_DIR}/service_account.json'
os.environ['WORK_DIR'] = WORK_DIR
os.environ['OUTPUT_DIR'] = f'{WORK_DIR}/output'
os.environ['AVATAR_IMAGE_PATH'] = f'{WORK_DIR}/assets/images/avatar.png'
os.environ['SADTALKER_DIR'] = '/content/SadTalker'
os.environ['DRIVE_OUTPUT_FOLDER'] = config.get('drive_output_folder', f'{DRIVE_BASE}/videos')

# Engine config
os.environ['OLLAMA_HOST'] = 'http://localhost:11434'
os.environ['OLLAMA_MODEL'] = 'gemma3:4b'
os.environ['TTS_ENGINE'] = 'edge'
os.environ['IMAGE_ENGINE'] = 'sd'
os.environ['SD_MODEL'] = 'stabilityai/stable-diffusion-xl-base-1.0'
os.environ['ENABLE_FACE_ENHANCEMENT'] = 'true'
os.environ['WHISPER_MODEL_SIZE'] = 'base'
os.environ['SDXL_INFERENCE_STEPS'] = '25'

# Check avatar
avatar_path = os.environ['AVATAR_IMAGE_PATH']
if not os.path.exists(avatar_path):
    for ext in ['png', 'webp', 'jpg']:
        drive_avatar = f'{DRIVE_BASE}/avatar.{ext}'
        if os.path.exists(drive_avatar):
            os.makedirs(os.path.dirname(avatar_path), exist_ok=True)
            shutil.copy(drive_avatar, avatar_path)
            print(f'  ✅ Avatar copied from Drive ({ext})')
            break
    else:
        print('  ⚠️  Avatar not found!')
else:
    print(f'  ✅ Avatar found')

os.makedirs(os.environ['OUTPUT_DIR'], exist_ok=True)
os.makedirs(os.environ['DRIVE_OUTPUT_FOLDER'], exist_ok=True)

print(f'\n📋 TTS={os.environ["TTS_ENGINE"]} | SD={os.environ["SD_MODEL"].split("/")[-1]} | LLM={os.environ["OLLAMA_MODEL"]}')
print('✅ Environment ready!')

---
## ✅ Step 5 — Verify Everything

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 5: System check
# ═══════════════════════════════════════════════════════════════

import os, subprocess
os.chdir('/content/ai-yt-automation')

print('🔍 System Check\n')
checks = []

import torch
gpu = torch.cuda.is_available()
checks.append(('GPU', gpu, torch.cuda.get_device_name(0) if gpu else 'No GPU'))

try:
    r = subprocess.run(['ollama', 'list'], capture_output=True, text=True, timeout=5)
    checks.append(('Ollama', r.returncode == 0, 'Running'))
except: checks.append(('Ollama', False, 'Not running'))

try:
    r = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True, timeout=5)
    checks.append(('FFmpeg', r.returncode == 0, 'Installed'))
except: checks.append(('FFmpeg', False, 'Missing'))

for name, path in [
    ('Avatar', os.environ.get('AVATAR_IMAGE_PATH', '')),
    ('Credentials', os.environ.get('GOOGLE_SERVICE_ACCOUNT_FILE', '')),
    ('SadTalker', '/content/SadTalker/checkpoints'),
]:
    checks.append((name, os.path.exists(path), path.split('/')[-1]))

for pkg_name, imp in [('Edge TTS','edge_tts'),('Diffusers','diffusers'),('Whisper','whisper'),('MoviePy','moviepy')]:
    try: __import__(imp); checks.append((pkg_name, True, '✓'))
    except: checks.append((pkg_name, False, 'Missing'))

for name, ok, detail in checks:
    print(f'  {"✅" if ok else "❌"} {name:16s} {detail}')

failed = sum(1 for _, ok, _ in checks if not ok)
print(f'\n{"🚀 Ready!" if failed == 0 else f"⚠️ {failed} issue(s) — fix above"}')

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 5b: Check topic status
# ═══════════════════════════════════════════════════════════════

import gspread, os
from google.oauth2.service_account import Credentials

SCOPES = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file(
    os.environ['GOOGLE_SERVICE_ACCOUNT_FILE'], scopes=SCOPES
)
gc = gspread.authorize(creds)

sheet_url = os.environ.get('GOOGLE_SHEET_URL', '')
sheet_name = os.environ.get('GOOGLE_SHEET_NAME', 'Sheet1')

sh = gc.open_by_url(sheet_url)
ws = sh.worksheet(sheet_name)
records = ws.get_all_records()

print(f'📊 Found {len(records)} topics:\n')
for i, row in enumerate(records):
    status = row.get('Status', 'Unknown')
    topic = row.get('Topic', row.get('Kural', 'N/A'))[:50]
    emoji = {'Pending': '⏳', 'Processing': '🔄', 'Done': '✅', 'Failed': '❌'}.get(status, '❓')
    print(f'  {emoji} Row {i+2}: [{status}] {topic}')

pending = sum(1 for r in records if r.get('Status') == 'Pending')
print(f'\n📋 Pending topics: {pending}')
if pending == 0:
    print('\n⚠️  No pending topics! Add rows with Status=Pending or reset below.')

---
## 🎬 Step 6 — Full Pipeline

Runs all 11 steps: Topic → Story → SEO → 5 Images → TTS → Avatar → Subtitles → Video → YouTube → Backup

> ⚠️ Make sure you have topics with Status = `Pending` in your Sheet!

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 6: Run Full Pipeline
# ═══════════════════════════════════════════════════════════════

import os, glob
os.chdir('/content/ai-yt-automation')

!python -m ai_shorts run --mode full

# Show result
videos = sorted(glob.glob('/content/ai-yt-automation/output/*/final_video.mp4'))
if videos:
    print(f'\n✅ Pipeline completed successfully!')
    print(f'🎬 Video: {videos[-1]}')
else:
    wavs = sorted(glob.glob('/content/ai-yt-automation/output/*/voice.wav'))
    if wavs:
        print(f'\n⚠️  Pipeline stopped early — voice generated but no video.')
        print(f'    Check logs above for errors.')
    else:
        print(f'\n❌ Pipeline failed — check logs above.')

---
## 📺 Step 7 — Preview & Download

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 7: Preview + download
# ═══════════════════════════════════════════════════════════════

from IPython.display import Video, display, Image
from pathlib import Path
import glob, shutil, os

output_dir = Path('/content/ai-yt-automation/output')
DRIVE_BASE = '/content/drive/MyDrive/ai-youtube-automation'

scenes = sorted(glob.glob(str(output_dir / '*/scenes/scene_*.png')))
if scenes:
    print(f'🖼️  {len(scenes)} Scene Images:\n')
    for img in scenes:
        display(Image(filename=img, width=250))

videos = sorted(glob.glob(str(output_dir / '*/final_video.mp4')))
if videos:
    print(f'\n🎬 Final Video:')
    display(Video(videos[-1], width=360))
    drive_out = os.environ.get('DRIVE_OUTPUT_FOLDER', f'{DRIVE_BASE}/videos')
    os.makedirs(drive_out, exist_ok=True)
    shutil.copy(videos[-1], f'{drive_out}/latest_video.mp4')
    print(f'💾 Saved to: {drive_out}/latest_video.mp4')
else:
    print('⚠️  No video found — run the Full Pipeline (Step 6) first.')

---
---

# 🔧 Advanced / Optional Cells

> **⚠️ Everything below is OPTIONAL.** The pipeline above runs everything with "Run All".
> These cells are for debugging, resetting topics, or one-time YouTube setup.


---
## 📊 Reset Topics (if needed)

Reset Failed/Processing topics back to `Pending`.

In [ ]:
# ═══════════════════════════════════════════════════════════════
# OPTIONAL: Reset Failed/Processing topics back to Pending
# ═══════════════════════════════════════════════════════════════
# ⚠️ Only run this if you want to re-process failed/stuck topics!

import gspread, os
from google.oauth2.service_account import Credentials

SCOPES = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file(
    os.environ['GOOGLE_SERVICE_ACCOUNT_FILE'], scopes=SCOPES
)
gc = gspread.authorize(creds)

sheet_url = os.environ.get('GOOGLE_SHEET_URL', '')
sheet_name = os.environ.get('GOOGLE_SHEET_NAME', 'Sheet1')

sh = gc.open_by_url(sheet_url)
ws = sh.worksheet(sheet_name)

header = ws.row_values(1)
try:
    status_col = header.index('Status') + 1
except ValueError:
    print('❌ No "Status" column found in row 1')
    status_col = None

if status_col:
    all_status = ws.col_values(status_col)
    reset_count = 0
    for i, status in enumerate(all_status[1:], start=2):
        if status in ('Failed', 'Processing'):
            ws.update_cell(i, status_col, 'Pending')
            reset_count += 1
            print(f'  🔄 Row {i}: {status} → Pending')
    
    if reset_count > 0:
        print(f'\n✅ Reset {reset_count} topic(s) to Pending!')
    else:
        print('ℹ️  No Failed/Processing topics to reset')

In [ ]:
# Download to your computer
from google.colab import files
import glob
videos = sorted(glob.glob('/content/ai-yt-automation/output/*/final_video.mp4'))
if videos: files.download(videos[-1])

---
## 🔑 One-Time: YouTube OAuth Token

Run only ONCE to generate your YouTube refresh token.

1. Run the cell below
2. Click the URL it prints
3. Authorize with your Google account
4. Copy the authorization code back into Colab
5. Save the printed tokens to your `config.json`

In [ ]:
# ═══════════════════════════════════════════════════════════════
# ONE-TIME: Generate YouTube refresh token
# ═══════════════════════════════════════════════════════════════

!pip install google-auth-oauthlib google-api-python-client -q

import os

DRIVE_BASE = '/content/drive/MyDrive/ai-youtube-automation'
secret = '/content/ai-yt-automation/client_secret.json'
if not os.path.exists(secret):
    secret = f'{DRIVE_BASE}/client_secret.json'

from google_auth_oauthlib.flow import InstalledAppFlow

flow = InstalledAppFlow.from_client_secrets_file(
    secret, ['https://www.googleapis.com/auth/youtube.upload']
)

creds = flow.run_console()

print('\n' + '=' * 50)
print('🔑 Add these to config.json in Drive:')
print('=' * 50)
print(f'"youtube_client_id": "{creds.client_id}",')
print(f'"youtube_client_secret": "{creds.client_secret}",')
print(f'"youtube_refresh_token": "{creds.refresh_token}"')
print('\n✅ Copy the values above into your config.json!')

---
## 🧪 Test Run (story + images + audio only)

> ⚠️ **Test mode stops early** — it does NOT run SadTalker, video composition, or YouTube upload.
> Use **Step 6 (Full Pipeline)** for the complete video.

In [ ]:
# ═══════════════════════════════════════════════════════════════
# TEST: story → 5 images → audio ONLY (no video/upload)
# ═══════════════════════════════════════════════════════════════

import os
os.chdir('/content/ai-yt-automation')
!python -m ai_shorts run --mode test

---

## 📝 config.json Template

```json
{
  "google_sheet_url": "https://docs.google.com/spreadsheets/d/YOUR_SHEET_ID/edit",
  "google_sheet_name": "Sheet1",
  "youtube_client_id": "YOUR_CLIENT_ID.apps.googleusercontent.com",
  "youtube_client_secret": "GOCSPX-your-secret",
  "youtube_refresh_token": "1//your-refresh-token",
  "telegram_bot_token": "1234567890:AAxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
  "telegram_chat_id": "123456789",
  "auto_upload_youtube": true,
  "video_privacy": "public",
  "drive_output_folder": "/content/drive/MyDrive/ai-youtube-automation/videos"
}
```

## 🛠️ Troubleshooting

| Issue | Fix |
|-------|-----|
| `No pending topics` | Add rows with Status=`Pending` in Sheet |
| `CUDA out of memory` | Runtime → Restart runtime → Run all |
| `Ollama connection refused` | Re-run Step 3 |
| `Avatar not found` | Upload avatar.png to Drive |
| `YouTube token expired` | Re-run OAuth cell above |

---

**Built with Clean Architecture ❤️ — Your AI Employee Working 24/7**